In [65]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [66]:
df = pd.read_csv("data\911.csv")

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 663522 entries, 0 to 663521
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   lat        663522 non-null  float64
 1   lng        663522 non-null  float64
 2   desc       663522 non-null  object 
 3   zip        583323 non-null  float64
 4   title      663522 non-null  object 
 5   timeStamp  663522 non-null  object 
 6   twp        663229 non-null  object 
 7   addr       663522 non-null  object 
 8   e          663522 non-null  int64  
dtypes: float64(3), int64(1), object(5)
memory usage: 45.6+ MB


In [68]:
df = df.drop_duplicates(inplace=False)
df.shape[0]

663282

In [69]:
df.isnull().sum()

lat              0
lng              0
desc             0
zip          80165
title            0
timeStamp        0
twp            293
addr             0
e                0
dtype: int64

In [70]:
df['coords'] = list(zip(df['lat'], df['lng']))

df = df.drop(['lat', 'lng'], axis=1)
df = df[['coords', 'desc', 'zip', 'title', 'timeStamp', 'twp', 'addr', 'e']]

In [71]:
nullBothCol = df[(df['zip'].isnull()) & (df['twp'].isnull())]
nullBothCol.shape[0]

169

In [72]:
df = df.drop(nullBothCol.index, axis=0)
df.shape[0]

663113

In [73]:
nullZip = df[df['zip'].isnull()]
nullZip.shape[0]

79996

In [74]:
df = df.drop(nullZip.index, axis=0)
df.shape[0]

583117

In [75]:
nullTwp = df[df['twp'].isnull()]
nullTwp.shape[0]

124

In [76]:
df = df.drop(nullTwp.index, axis=0)
df.shape[0]

582993

In [77]:
def find_best_twp(row, df, options):
    best_twp = None
    best_cols = None
    best_size = float('inf')

    for r in range(1, len(options) + 1):
        for subset in itertools.combinations(options, r):
            cur_df = df
            for col in subset:
                cur_df = cur_df[cur_df[col] == row[col]]

            unique_twps = cur_df['twp'].unique()

            if 0 < len(unique_twps) < best_size:
                best_size = len(unique_twps)
                best_twp = unique_twps
                best_cols = subset

    if best_twp is not None:
        return {
            'cols_used': list(best_cols),
            'twp': best_twp,
            'twp_count': best_size
        }

    return

In [78]:
results = []

for idx in nullTwp.index:
    res = find_best_twp(nullTwp.loc[idx], df, ['coords', 'zip', 'addr'])
    if res:
        res['row_index'] = idx
        results.append(res)
    else:
        nullTwp.drop(idx, axis=0, inplace=True)

stats_df = pd.DataFrame(results).set_index('row_index')

In [79]:
nullTwp.shape[0]

122

In [89]:
stats_df.sample(5)

,cols_used,twp,twp_count
row_index,,,
646783,"[zip, addr]","[UPPER PROVIDENCE, PLYMOUTH]",2
579857,[zip],"[POTTSTOWN, LOWER POTTSGROVE, LIMERICK, WEST P...",33
658479,[zip],"[HATFIELD TOWNSHIP, LANSDALE, UPPER GWYNEDD, T...",29
77069,[addr],"[PENNSBURG, NORRISTOWN, SPRINGFIELD, TOWAMENCI...",6
474764,[coords],[UPPER PROVIDENCE],1


In [81]:
stats_df[stats_df['twp_count'] == 1].shape[0]

63

In [91]:
for idx, row in stats_df.iterrows():
    if row['twp_count'] == 1:
        nullTwp.loc[idx, 'twp'] = row['twp'][0]
        
    else:
        cols = row['cols_used']
        filtered_df = df.copy()
        for col in cols:
            filtered_df = filtered_df[filtered_df[col] == nullTwp.loc[idx, col]]

        target_coords = np.array(nullTwp.loc[idx, 'coords'])  
        points = np.vstack(filtered_df['coords'].to_numpy())
        dists = np.linalg.norm(points - target_coords, axis=1)
        nearest_idx = np.argmin(dists)
        
        nullTwp.loc[idx, 'twp'] = filtered_df.iloc[nearest_idx]['twp']

In [92]:
nullTwp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 122 entries, 1635 to 661678
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   coords     122 non-null    object 
 1   desc       122 non-null    object 
 2   zip        122 non-null    float64
 3   title      122 non-null    object 
 4   timeStamp  122 non-null    object 
 5   twp        122 non-null    object 
 6   addr       122 non-null    object 
 7   e          122 non-null    int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 12.6+ KB
